# This notebook is a Johdanto datatieteeseen 2022 project. 

## Collecting data

In this project, I'm going to use data from Finnish car dealing website Autotalli.com. With this data, I'm going to create a ML model to predict the sales price for particular car. With this in mind, consumer or car dealer can check the valid price for a car. This is particularly interesting, because due to the component crisis, car markets are not saturated. The demand for cars is higher than supply, and therefore the prices have gone up. Before there used to be around 100k cars on sale in nettiauto.com, but nowadays there are about 80k sales notes. I first tried to scrape information from there, but the page loaded a dummy website because it recognised scraping.

In [74]:
#Import neede libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as rq
import math


In [76]:
#Scrape data from Autotalli.com
url = "https://www.autotalli.com/vaihtoautot/listaa/sivu/"

#Dataset for scraped data. This dataset is saved into .csv file as a backup, so no need to scrape multiple times if an error with later coding is occurred.
raw_data = {
    'Make': [],
    'Model': [],
    'Odometer': [],
    'Sales price': []
}

In [77]:
#BS object
autotalli_page = rq.get(url)
soup = bs(autotalli_page.content, 'html.parser')
#There are 20 cars displayed in each page, so number of pages is number of cars/cars per page, rounded up.
intMaxPages = math.ceil(int(soup.find('span', class_="yhteensa").text)/20)

#Loop through all pages
for page in range(1,intMaxPages+1):
    #BS object
    url = url + str(page)
    autotalli_page = rq.get(url)
    soup = bs(autotalli_page.content, 'html.parser')
    salesNotes = soup.find_all("div", class_="carsListItemDetailsContainer")
    counter = 0

    #Loop through all notes in one page
    for note in salesNotes:
        details = note.find(class_="carsListItemCarNameContainer").find(class_="carsListItemNameLink").text
        #Take car make and model out of details
        details_splitted = details.split()

        make = details_splitted[0]
        model = details_splitted[1]

        price = note.find(class_="carsListItemCarPrice").find(class_="carsListItemCarBottomContainerItem").text
        odo = note.find(class_="usedCarsListItemCarMeterReading").find(class_="carsListItemCarBottomContainerItem").text

        #Add data to vector
        raw_data['Make'].append(make)
        raw_data['Model'].append(model)
        raw_data['Odometer'].append(odo)
        raw_data['Sales price'].append(price)
        
#Write vector to pd dataframe        
dfRaw = pd.DataFrame(raw_data)
print("Data scraped with total of", len(dfRaw), "rows")

Data scraped with total of 43720


In [84]:
#Write the data to .csv file
dfRaw.to_csv("raw_data.csv")

## 

## Data processing

The data itself is in pretty nice format, because the scraper was implemented for this website only.

In [99]:
#Load the csv file to Pandas dataframe and see what've got
data = pd.read_csv('raw_data.csv')

#Some basic information about the data
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43720 entries, 0 to 43719
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   43720 non-null  int64 
 1   Make         43720 non-null  object
 2   Model        43720 non-null  object
 3   Odometer     43720 non-null  int64 
 4   Sales price  43720 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.7+ MB


,Unnamed: 0,Make,Model,Odometer,Sales price
0,0,Mercedes-Benz,S,22000,91800
1,1,Ford,Edge,101000,31800
2,2,BMW,iX3,4000,72800
3,3,BMW,745,26000,78800
4,4,Audi,RS3,79000,70800


The data seems like very clean. Let's remove the "Unnamed" column and search for irregularities in different columns.

In [128]:
data.drop(columns = ['Unnamed: 0'], inplace = True)
data.head()

KeyError: "['Unnamed: 0'] not found in axis"

In [101]:
print(data['Make'].unique())

['Mercedes-Benz' 'Ford' 'BMW' 'Audi' 'Volkswagen' 'Skoda' 'Toyota' 'Mazda'
 'Mitsubishi' 'Renault' 'Kia' 'Nissan' 'Peugeot' 'Fiat' 'Honda' 'Daewoo'
 'Chevrolet' 'Saab' 'Chrysler' 'Volvo' 'Dacia' 'Jaguar' 'Opel' 'Hyundai'
 'Land' 'Porsche']


In [102]:
print(data['Model'].unique())

['S' 'Edge' 'iX3' '745' 'RS3' 'Crafter' 'Fiesta' 'Fabia' 'Corolla'
 'Octavia' 'Focus' 'Passat' 'MPV' 'Lancer' '100' 'Laguna' '626' 'Grandis'
 'A6' 'Scénic' 'Cerato' 'Avensis' 'Primera' "cee'd" 'Bora' '307' 'Croma'
 'Mégane' 'Civic' '6' 'Evanda' 'Nubira' '9-5' 'Pt' 'V50' 'Picanto'
 'Twingo' 'Yaris' 'CR-V' 'Kuga' 'Logan' 'XF' 'CLK' 'C' 'Insignia' 'V70'
 'A4' 'S40' 'Punto' 'E' 'Astra' 'Sprinter' 'Scala' 'Eclipse' 'V40' 'V60'
 'Golf' 'XC60' 'Prius+' 'C-HR' 'Caddy' 'Qashqai' 'IONIQ' 'NV300' 'A' 'GLE'
 'Superb' 'X5' 'CLS' 'Rover' 'Taycan' 'e-tron']


In [110]:
print(data['Odometer'].unique())


[ 22000 101000   4000  26000  79000      0  10000   1000  15000      1
 224000 573000 276000 207000 343000 288000 384000 379000 243350 169000
 472000 213000 313000 265000 217000 225000 338000 304000 241000 179500
 212000 227000 127000 188000 355000 197000 111000 120000 147700 381000
 356000 106000 163000 270000 351000 231000 247000 216000 330000 220000
 107000 333000 239000 151000 113000 165000 118000  30000  60000  92000
 155000  29000 249000  94000  46000 105000  97000  40000  72000  53000
  57000  98000  95000  17000  11000   7000 139000   6000  88000  38000
 116000   9000  70000  14000  12000 420000]


Here we can see that there are actually rows with odometer value 0 or something very low. In this model, we want to get information about used cars. Let's decide that used cars have been driven for over 5000 km. 

In [119]:
#Drop all rows where can has been driven under 5000 km
for index, row in data.iterrows():
    if row['Odometer'] < 5000:
        data.drop(index, inplace = True)

In [120]:
print(data['Odometer'].unique())

[ 22000 101000  26000  79000  10000  15000 224000 573000 276000 207000
 343000 288000 384000 379000 243350 169000 472000 213000 313000 265000
 217000 225000 338000 304000 241000 179500 212000 227000 127000 188000
 355000 197000 111000 120000 147700 381000 356000 106000 163000 270000
 351000 231000 247000 216000 330000 220000 107000 333000 239000 151000
 113000 165000 118000  30000  60000  92000 155000  29000 249000  94000
  46000 105000  97000  40000  72000  53000  57000  98000  95000  17000
  11000   7000 139000   6000  88000  38000 116000   9000  70000  14000
  12000 420000]


In [121]:
print(data['Sales price'].unique())

[ 91800  31800  78800  70800    199    229    500    600    690    700
    740    800    900   2480   2490   6550   6580   6590   6600  22900
  35800  66800  92800  51800  32800 129800 124800  38780  45800  33800
 109800 159800    790]


In [122]:
#Drop all rows where Sales price in under 1000 euros
for index, row in data.iterrows():
    if row['Sales price'] < 1000:
        data.drop(index, inplace = True)

In [126]:
data.info()
data.head()
data.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10398 entries, 0 to 43704
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Make         10398 non-null  object
 1   Model        10398 non-null  object
 2   Odometer     10398 non-null  int64 
 3   Sales price  10398 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 406.2+ KB


Make           object
Model          object
Odometer        int64
Sales price     int64
dtype: object

Data is now cleaned so it can be saved to a new csv file

In [127]:
data.to_csv('processed_data.csv')

## Describing data

## Machine learning model